In [0]:
import pandas as pd
import iarray_spark as iaspark
import iarray as ia
import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

import pyspark.pandas as ps
import shutil

from time import time

In [0]:
shape = (512 * 1024, 1024)
chunks = (64 * 1024, 64)
blocks = (4 * 1024, 16)

dtype = np.float64

urlpath_a = "/iaspark_a.iarray"
urlpath_dbfs_a = "/dbfs/iaspark_a.iarray"
urlpath_out = "/iaspark_out.iarray"
urlpath_dbfs_out = "/dbfs/iaspark_out.iarray"

urlpath_parquet_a = "/iaspark_a.parquet"
urlpath_parquet_dbfs_a = "/dbfs/iaspark_a.parquet"
urlpath_parquet_out = "/iaspark_out.parquet"
urlpath_parquet_dbfs_out = "/dbfs/iaspark_out.parquet"

ia.set_config_defaults(chunks=chunks, blocks=blocks, dtype=dtype)

red = "max"
axis = 0

In [0]:
try:
    ia.remove_urlpath(urlpath_a)
except:
    pass
try:
    ia.remove_urlpath(urlpath_out)
except:
    pass
try:
    shutil.rmtree(urlpath_dbfs_a)
except:
    pass
try:
    shutil.rmtree(urlpath_dbfs_out)
except:
    pass


try:
    shutil.rmtree(urlpath_parquet_a)
except:
    pass
try:
    shutil.rmtree(urlpath_parquet_out)
except:
    pass
try:
    shutil.rmtree(urlpath_parquet_dbfs_a)
except:
    pass
try:
    shutil.rmtree(urlpath_parquet_dbfs_out)
except:
    pass

In [0]:
a = ia.random.uniform(shape, fp_mantissa_bits=4, urlpath=urlpath_a)
# a = ia.arange(shape, urlpath=urlpath_a)
shutil.copy(urlpath_a, urlpath_dbfs_a)

df = pd.DataFrame(a.data, columns=[str(i) for i in range(shape[1])])
pa_table = pa.Table.from_pandas(df)
# pq.write_table(pa_table, row_group_size=1024, where=urlpath_parquet_a)
pq.write_table(pa_table, where=urlpath_parquet_a, compression="LZ4")

shutil.copy(urlpath_parquet_a, urlpath_parquet_dbfs_a)

a.cratio

In [0]:
n = 2

In [0]:
import pyspark.sql.functions as f
try:
    shutil.rmtree(urlpath_parquet_dbfs_out)
except:
    pass
t = np.empty(n, dtype=np.float64)

for i in range(n):
    print(i)
    t0 = time()
    df_spark_ = spark.read.parquet(urlpath_parquet_a).repartition(16)
    df_spark = df_spark_.agg(*[f.avg(c).alias(c) for c in df_spark_.columns]).collect()
    # df_spark.write.parquet(urlpath_parquet_out)

    try:
        shutil.rmtree(urlpath_parquet_dbfs_out)
    except:
        pass

    t1 = time()
    t[i] = t1 - t0

print(f"time parquet (pyspark.DataFrame): {t[1:].mean():.2f} \u00B1 {t[1:].std():.2f} s")


In [0]:
import os
t = np.empty(n, dtype=np.float64)
for i in range(n):
    print(i)
    t0 = time()
    res = iaspark.mean(urlpath_dbfs_a, axis=axis, spark_context=sc)
    # res = iaspark.mean(urlpath_dbfs_a, axis=axis, spark_context=sc, urlpath=urlpath_out)
    # shutil.copy(urlpath_out, urlpath_dbfs_out)
    
    t1 = time()
    t[i] = t1 - t0
print(f"time iarray: {t[1:].mean():.2f} \u00B1 {t[1:].std():.2f} s")


In [0]:
a.cratio

In [0]:
res.data

In [0]:
try:
    shutil.rmtree(urlpath_a)
except:
    pass
try:
    shutil.rmtree(urlpath_out)
except:
    pass
try:
    shutil.rmtree(urlpath_dbfs_a)
except:
    pass
try:
    shutil.rmtree(urlpath_dbfs_out)
except:
    pass


try:
    shutil.rmtree(urlpath_parquet_a)
except:
    pass
try:
    shutil.rmtree(urlpath_parquet_out)
except:
    pass
try:
    shutil.rmtree(urlpath_parquet_dbfs_a)
except:
    pass
try:
    shutil.rmtree(urlpath_parquet_dbfs_out)
except:
    pass